# DataLoader

In [ ]:
class CustomDataset(Dataset):
    def init(self, txt_file, img_dir, coco_json_file, aug=False):
        """
        Inizializza il dataset personalizzato.
        Args:
        - txt_file: Il file di testo contenente i percorsi delle immagini.
        - img_dir: La cartella delle immagini.
        - coco_json_file: Il file JSON in formato COCO contenente le annotazioni.
        - aug: Booleano per attivare o meno l'augmentazione.
        """
        def generate_id(file_name):
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')
 
        with open(txt_file, 'r') as f:
            self.image_paths = [line.strip() for line in f.readlines()]
 
        self.img_dir = img_dir
 
        # Carica il file JSON delle annotazioni COCO
        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)
 
        # Crea una struttura per le annotazioni
        self.image_annotations = {}
        self.image_bboxes = {}
 
        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            category_id = annotation['category_id']
            bbox = annotation['bbox']  # Formato COCO [x_min, y_min, width, height]
 
            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []
 
            self.image_annotations[image_id].append(category_id)
            self.image_bboxes[image_id].append(bbox)
 
        # Mappa per associare ID immagine a file_name
        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        }
 
        # Trasformazioni di base e di augmentation
        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),   
        ])
 
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
 
        self.aug = aug
 
    def len(self):
        return len(self.image_paths)
 
    def getitem(self, index):
        # Estrai il nome dell'immagine e l'ID corrispondente
        img_name = os.path.basename(self.image_paths[index])
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', ''))
        if img_id not in self.image_info:
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")
        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size
        # Applica le trasformazioni
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)
        # Estrai le annotazioni e i bounding boxes
        categories = self.image_annotations.get(img_id, [])
        bboxes = self.image_bboxes.get(img_id, [])
        if not bboxes:  # Immagini senza annotazioni
            target = {
                "boxes": torch.zeros((0, 4), dtype=torch.float32),
                "labels": torch.zeros((0,), dtype=torch.int64)
            }
        else:
            # Converte da formato COCO [x_min, y_min, width, height] a [x_min, y_min, x_max, y_max]
            scale_x = 320 / original_width
            scale_y = 320 / original_height
            scaled_bboxes = [
                torch.tensor([  # [x1, y1, x2, y2]
                    bbox[0] * scale_x,               # x_min
                    bbox[1] * scale_y,               # y_min
                    (bbox[0] + bbox[2]) * scale_x,   # x_max
                    (bbox[1] + bbox[3]) * scale_y    # y_max
                ], dtype=torch.float32)
                for bbox in bboxes
            ]

In [ ]:
def collate_fn(batch):
    """
    Funzione di collation per il DataLoader, utile per il batching di immagini e annotazioni.
    La funzione restituirà un batch di immagini e un batch di target, formattato correttamente per Faster R-CNN.
    
    Args:
    - batch: lista di tuple (image, target)
    
    Returns:
    - images: batch di immagini
    - targets: lista di dizionari contenenti le annotazioni per ogni immagine
    """
    # Separa immagini e target
    images, targets = zip(*batch)

    # Converte la lista di immagini in un batch di immagini
    images = list(images)
    
    # Varia le dimensioni delle immagini per garantire che siano tutte della stessa dimensione
    # La dimensione di uscita deve essere la stessa per tutte le immagini del batch
    images = [F.resize(img, (320, 320)) for img in images]  # Assumendo che 320 sia la dimensione target

    # Restituisci il batch
    return images, list(targets)

In [ ]:
# Creazione dei dataset
train_dataset_frcc = CustomDataset(train_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=True)
valid_dataset_frcc = CustomDataset(val_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=False)  
test_dataset_frcc = CustomDataset(test_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=False)  

# Creazione dei DataLoader
train_loader_frcc = DataLoader(train_dataset_frcc, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader_frcc = DataLoader(valid_dataset_frcc, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader_frcc = DataLoader(test_dataset_frcc, batch_size=32, shuffle=False, collate_fn=collate_fn)

## Check DataLoader

In [ ]:
# Numero totale di campioni per ogni DataLoader
train_size = len(train_loader_frcc.dataset)
val_size = len(val_loader_frcc.dataset)
test_size = len(test_loader_frcc.dataset)

# Numero di batch per ogni DataLoader
train_batches = len(train_loader_frcc)
val_batches = len(val_loader_frcc)
test_batches = len(test_loader_frcc)

# Visualizza i risultati
print(f"Numero totale di elementi nel train_loader: {train_size}")
print(f"Numero totale di batch nel train_loader: {train_batches}")
print(f"Numero totale di elementi nel val_loader: {val_size}")
print(f"Numero totale di batch nel val_loader: {val_batches}")
print(f"Numero totale di elementi nel test_loader: {test_size}")
print(f"Numero totale di batch nel test_loader: {test_batches}")

# Somma totale degli elementi nei DataLoader
total_elements = train_size + val_size + test_size
print(f"Numero totale di elementi in tutti i DataLoader: {total_elements}")

In [ ]:
def check_txt_vs_json(txt_paths, coco_data):
    """
    Controlla se le immagini del JSON sono presenti in almeno uno dei file TXT.
    
    Args:
        txt_paths (list): Lista di percorsi ai file TXT.
        coco_data (dict): Dati in formato COCO.
    """
    # Estrai i nomi delle immagini dal JSON
    image_names = [image['file_name'] for image in coco_data['images']]
    
    # Inizializza un set per contenere tutte le immagini presenti nei TXT
    txt_image_names = set()
    
    # Leggi i nomi delle immagini da ciascun file TXT
    for txt_path in txt_paths:
        with open(txt_path, 'r') as f:
            txt_image_names.update(os.path.basename(line.strip()) for line in f.readlines())
    
    # Trova le immagini presenti nel JSON ma non in nessuno dei TXT
    missing_in_txts = [name for name in image_names if name not in txt_image_names]
    
    # Verifica e stampa i risultati
    print("\nControllo completato:")
    if missing_in_txts:
        print(f"Errore: le seguenti immagini non sono presenti in nessuno dei file TXT forniti:\n{missing_in_txts}")
    else:
        print("Tutte le immagini del JSON sono presenti in almeno uno dei file TXT.")

In [ ]:
txt_paths = [train_txt_pth, val_txt_pth, test_txt_pth]

In [ ]:
check_txt_vs_json(txt_paths, coco_data)

In [ ]:
# Verifica il DataLoader
def check_dataloader(dataloader, num_batches=5):
    for i, (images, targets) in enumerate(dataloader):
        # Check numero di batch
        if i >= num_batches:
            break
        
        # Verifica che le immagini siano nel formato corretto [N, C, H, W]
        print(f"Batch {i + 1}:")
        print(f"  Numero di immagini: {len(images)}")
        print(f"  Dimensioni dell'immagine: {images[0].shape}")  # Forma della prima immagine nel batch
        
        # Verifica che i target siano nel formato corretto
        print(f"  Numero di target: {len(targets)}")
        print(f"  Bounding boxes: {targets[0]['boxes'].shape}")  # Dimensione dei bounding boxes
        print(f"  Etichette: {targets[0]['labels'].shape}")  # Dimensione delle etichette
        
        # Check che i bounding boxes abbiano il formato corretto
        for target in targets:
            assert target['boxes'].shape[1] == 4, "Le dimensioni dei bounding box non sono corrette!"
            assert target['labels'].dtype == torch.int64, "Le etichette non sono di tipo int64!"
        
        print("  -----")

def visualize_batch(images, targets, num_images=3):
    for i in range(min(num_images, len(images))):
        # Converti la prima immagine del batch in numpy (per matplotlib)
        image = images[i].cpu().numpy().transpose(1, 2, 0)  # [C, H, W] -> [H, W, C]
        image = np.clip(image, 0, 1)  # Assicurati che i valori siano nel range [0, 1]

        # Ottieni i bounding boxes
        boxes = targets[i]['boxes'].cpu().numpy()
        labels = targets[i]['labels'].cpu().numpy()

        # Visualizza l'immagine
        plt.figure(figsize=(6, 6))
        plt.imshow(image)
        
        # Aggiungi i bounding boxes
        for box, label in zip(boxes, labels):
            x_min, y_min, x_max, y_max = box
            plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                              linewidth=2, edgecolor='r', facecolor='none'))
            plt.text(x_min, y_min, f"Class: {label}", color='red', fontsize=12)
        
        plt.show()

In [ ]:
check_dataloader(train_loader_frcc)
check_dataloader(val_loader_frcc)
check_dataloader(test_loader_frcc)

# Visualizza alcune immagini con i bounding box
for images, targets in train_loader_frcc:
    visualize_batch(images, targets)
    break  # Visualizza solo il primo batch

# Modello Faster R-CNN (Resnet50)

In [ ]:
from torchvision import models

from torchvision.models.detection import fasterrcnn_resnet50_fpn

from torchvision.models.detection import FasterRCNN

from torch import optim
 
# Carica il modello Faster R-CNN con ResNet50 e FPN

model = fasterrcnn_resnet50_fpn(pretrained=True)
 
num_classes = 12  # 11 classi + 1 per il background
 
# Modifica il numero di classi in output

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
 
# Imposta il dispositivo (GPU o CPU)

device = torch.device('cuda')
 
# Definisci l'ottimizzatore (esempio con AdamW)

optimizer = optim.AdamW(model.parameters(), lr=1e-4)
 
# Esegui un ciclo di addestramento

num_epochs = 10  # Modifica a seconda del tuo caso

for epoch in range(num_epochs):

    model.train()  # Imposta il modello in modalità training

    total_loss = 0  # Per monitorare la perdita totale per epoca
 
    for images, targets in train_loader:

        images = [image.to(device) for image in images]  # Trasferisci le immagini al dispositivo

        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]  # Trasferisci i target al dispositivo
 
        # Ottimizza il modello

        loss_dict = model(images, targets)  # Calcola le perdite

        losses = sum(loss for loss in loss_dict.values())  # Somma le perdite
 
        # Ottimizza

        optimizer.zero_grad()

        losses.backward()

        optimizer.step()
 
        total_loss += losses.item()
 
    print(f"Epoca {epoch + 1}/{num_epochs}, Perdita Totale: {total_loss}")
 
    # Valutazione dopo ogni epoca

    model.eval()  # Imposta il modello in modalità valutazione

    with torch.no_grad():

        for images, targets in val_loader:

            images = [image.to(device) for image in images]

            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Calcola le predizioni per la valutazione

            prediction = model(images)
 
    # Salva il modello dopo ogni epoca (opzionale)

    torch.save(model.state_dict(), f"model_epoch_{epoch}.pth")